# Tutorial 00: P1 Finite Elements for the Poisson Equation

This is still Work in Progress. I will add the proper documentation when I have a fully functional version.

In [ ]:
#include<dune/jupyter.hh>
#include"poissonp1.hh"

In [ ]:
Dune::ParameterTree ptree;
Dune::ParameterTreeParser::readINITree("tutorial00.ini", ptree);

In [ ]:
ptree

In [ ]:
const int dim = 2;
using Grid = Dune::UGGrid<dim>;
std::string filename = ptree.get("grid.twod.filename", "unitsquare.msh");
Dune::GridFactory<Grid> factory;
Dune::GmshReader<Grid>::read(factory,filename,true,true);
std::unique_ptr<Grid> grid(factory.createGrid());
grid->globalRefine(ptree.get<int>("grid.refinement"));

In [ ]:
grid

In [ ]:
using DF = Grid::ctype;
using RF = double;
auto gv = grid->leafGridView();
using GV = decltype(gv);

In [ ]:
typedef Dune::PDELab::PkLocalFiniteElementMap<GV,DF,RF,1> FEM;
FEM fem(gv);
typedef Dune::PDELab::ConformingDirichletConstraints CON;
typedef Dune::PDELab::ISTL::VectorBackend<> VBE;
typedef Dune::PDELab::GridFunctionSpace<GV,FEM,CON,VBE> GFS;
GFS gfs(gv,fem);
gfs.name("P1");

In [ ]:
using Z = Dune::PDELab::Backend::Vector<GFS,RF>;
Z z(gfs);

The nesting of the following block is *currently* necessary because of a bug in `xeus-cling` that does not allow the definition of lambdas in the context of `main`. I will report this upstream and check for a solution once I have the main stuff working.

In [ ]:
{
    auto g = Dune::PDELab::makeGridFunctionFromCallable(
        gv,
        [](const auto& x){
            RF s=0.0;
            for (std::size_t i=0; i<x.size(); i++)
                s+=x[i]*x[i];
            return s;
        }
    );
    Dune::PDELab::interpolate(g,gfs,z);
}

In [ ]:
typedef typename GFS::template ConstraintsContainer<RF>::Type CC;
CC cc;

In [ ]:
{
  auto b = Dune::PDELab::makeBoundaryConditionFromCallable(
      gv,
      [](const auto& x){return true;}
  );
  Dune::PDELab::constraints(b, gfs, cc);
  std::cout << "constrained dofs=" << cc.size() << " of "
            << gfs.globalSize() << std::endl;
}

The `LocalOperator` has a problem. It gets passed a grid function constructed from a lambda. Not yet sure how that works with the `xeus-cling` bug.